In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')
# sys.path.append('/home/ec2-user/SageMaker/PETs-Experiment')
# os.chdir('/home/ec2-user/SageMaker/PETs-Experiment/demo')
!pip install -r ..\requirements.txt

import PETs_Experiment

In [2]:
# loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[Adt Income] adult.csv'
#                                ,na_values = {k : '?' for k in ['workclass'
#                                                               ,'occupation'
#                                                               ,'native-country'
#                                                               ]
#                                             }
#                                )
loader = PETs_Experiment.Loader(filepath = '../[sunset]/data/[NHANES] B.csv'
                               ,header_exist = False
                               ,header_names = ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                               )
print(loader.data.head(1))

    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  


In [3]:
splitter = PETs_Experiment.Splitter(data              = loader.data
                                   ,num_samples       = 30
                                   ,train_split_ratio = 0.8)
print(splitter.data[1]['train'     ].shape[0])
print(splitter.data[1]['validation'].shape[0])
print(splitter.data[1]['train'     ].head(1) )
print(splitter.data[1]['validation'].head(1) )

3352
838
    gen   age   race       edu      mar        bmi  dep  pir   gh  mets  qm  \
0  Male  62.0  White  Graduate  Married  27.799999    0    0  0.0   0.0  Q2   

   dia  
0    1  
      gen   age     race      edu       mar        bmi  dep  pir   gh  mets  \
0  Female  38.0  Mexican  College  Divorced  29.299999    0    1  0.0   0.0   

   qm  dia  
0  Q3    0  


In [4]:
preproc = PETs_Experiment.Preprocessor(
     data = splitter.data[1]['train']
    ,missing=True
        ,missing_method  = 'drop'
        ,missing_columns = None
    ,outlier=True
        ,outlier_method  = 'IQR'
        # ,outlier_columns = {'ignore': ['hours-per-week']}
        ,outlier_columns = {'ignore': ['dep','pir','gh','mets','dia']}
    ,encoding=True
        ,encoding_method  = 'Label'
        ,encoding_columns = None
    ,scaling=True
        # ,scaling_method  = 'Standard'
        ,scaling_method  = 'MinMax'
        # ,scaling_columns = {'focus' : 'fnlwgt'}
        ,scaling_columns = {'focus' : ['age','bmi']}
)
print('\n\n\n')
print(preproc.data.head(1))
print('\n\n\n')
print(preproc.encoder)
print('\n\n\n')
print(preproc.scaler)

Preprocessor - Outlierist (IQR): No rows have been dropped on age.


Preprocessor - Outlierist (IQR): Dropped   96 rows on bmi . Kept [12.100001335144043, 44.899996757507324] only.
Preprocessor - Outlierist (IQR): Totally Dropped   96 in 3352 rows.
Preprocessor - Encoder (Label): Column mar  been labelized from 0 to 5.
Preprocessor - Encoder (Label): Column gen  been labelized from 0 to 1.
Preprocessor - Encoder (Label): Column race been labelized from 0 to 4.
Preprocessor - Encoder (Label): Column edu  been labelized from 0 to 4.
Preprocessor - Encoder (Label): Column qm   been labelized from 0 to 3.
Preprocessor - Scaler (MinMax): Column bmi been standardized.
Preprocessor - Scaler (MinMax): Column age been standardized.




   gen  age  race  edu  mar       bmi  dep  pir   gh  mets  qm  dia
0    1  0.7     4    3    1  0.440397    0    0  0.0   0.0   1    1




{'mar': LabelEncoder(), 'gen': LabelEncoder(), 'race': LabelEncoder(), 'edu': LabelEncoder(), 'qm': LabelEncoder()}




{'bmi': MinMaxScaler(), 'age': MinMaxScaler()}


In [9]:
synthesizer = PETs_Experiment.Synthesizer(data = preproc.data
                                         ,synthesizing_method = 'sdv-singletable-gaussiancoupula'
                                         )
synthesizer.fit_sample()
print(synthesizer.data_syn.head(1))

Synthesizer (SDV - SingleTable): Metafile loading time: 0.013 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'age'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'bmi'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 2.05 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 3256 rows (same as raw) in 0.2998 sec.
   gen      age  race  edu  mar       bmi  dep  pir   gh  mets  qm  dia
0    1  0.92813     3    4    1  0.812611    0    1  0.0   0.0   3    0


In [10]:
postproc = PETs_Experiment.Postprocessor(data = synthesizer.data_syn
                                        ,encoder = getattr(preproc ,'encoder' ,None)
                                        ,scaler  = getattr(preproc ,'scaler'  ,None)
                                        )
print(postproc.data.head(1))

Postprocessor - Scaler (MinMaxScaler): Decoding bmi.
Postprocessor - Scaler (MinMaxScaler): Decoding age.
Postprocessor - Encoder (LabelEncoder): Decoding mar.
Postprocessor - Encoder (LabelEncoder): Decoding gen.
Postprocessor - Encoder (LabelEncoder): Decoding race.
Postprocessor - Encoder (LabelEncoder): Decoding edu.
Postprocessor - Encoder (LabelEncoder): Decoding qm.
    gen       age   race         edu      mar        bmi  dep  pir   gh  mets  \
0  Male  75.68779  Other  HighSchool  Married  39.040852    0    1  0.0   0.0   

   qm  dia  
0  Q4    0  


In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment


para_Executor = {
     'Loader_filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
    ,'Loader_na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
    ,'Splitter_num_samples'       : 2
    ,'Splitter_train_split_ratio' : 0.8
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    ,'Preprocessor_scaling'         : True
    ,'Preprocessor_scaling_method'  : 'Standard'
    ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer_synthesizing_method' : 'sdv-singletable-gaussiancoupula'
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run_single_process()
executor.run()


for k ,v in executor.loader.items():
    print(k)
    print(v.data.head(1))

Executor - Loader: [Adt Income] adult.csv loading time: 9.4142 sec.
Executor - Splitter: Default splitting time: 0.087 sec.
Preprocessor - Outlierist (IQR): Dropped   236 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  3058 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   204 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1711 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1045 rows on fnlwgt         . Kept [-63828.75, 419397.25] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6046 in 36138 rows.
Preprocessor - Encoder (Label): Column marital-status  been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocessor - Encoder (Label): Column workclass       been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column relationship    been labelized

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.1571 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30092 rows (same as raw) in 1.3548 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 18.4659 sec.
Preprocessor - Outlierist (IQR): Dropped   238 rows on educational-num. Kept [3.0, 19.0] only.
Preprocessor - Outlierist (IQR): Dropped  3007 rows on capital-gain   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped   215 rows on age            . Kept [-0.5, 75.5] only.
Preprocessor - Outlierist (IQR): Dropped  1716 rows on capital-loss   . Kept [0.0, 0.0] only.
Preprocessor - Outlierist (IQR): Dropped  1074 rows on fnlwgt         . Kept [-63007.625, 418921.375] only.
Preprocessor - Outlierist (IQR): Totally Dropped  6051 in 36232 rows.
Preprocessor - Encoder (Label): Column marital-status  been labelized from 0 to  6.
Preprocessor - Encoder (Label): Column gender          been labelized from 0 to  1.
Preprocess

c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'fnlwgt'. Data will not be rounded.
  warnings.warn(


Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 14.2025 sec.
Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30181 rows (same as raw) in 1.3259 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 15.5614 sec.
====== ====== ====== ====== ====== ======
Executor: Total execution time: 44.8229 sec.
====== ====== ====== ====== ====== ======


...

Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 16.8628 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula spent 17.0154 sec.


Synthesizing:  50%|█████     | 1/2 [00:18<00:18, 18.36s/it]

Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30111 rows (same as raw) in 1.3191 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 18.2539 sec.


Synthesizing: 100%|██████████| 2/2 [00:18<00:00,  9.43s/it]

Synthesizer (SDV - SingleTable): Sampling GaussianCoupula # 30111 rows (same as raw) in 1.4452 sec.
Executor - Synthesizer: sdv-singletable-gaussiancoupula synthesizing time: 18.7308 sec.
====== ====== ====== ====== ====== ======
Executor: Total execution time: 32.519 sec.
====== ====== ====== ====== ====== ======
[Adt Income] adult.csv
   age workclass  fnlwgt education  educational-num marital-status  \
0   25   Private  226802      11th                7  Never-married   

          occupation relationship   race gender  capital-gain  capital-loss  \
0  Machine-op-inspct    Own-child  Black   Male             0             0   

   hours-per-week native-country income  
0              40  United-States  <=50K  


In [1]:
import os
import sys

sys.path.append('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment')
os.chdir('D:\\Dropbox\\89_其他應用\\GitHub\\PETs-Experiment\\demo')

import PETs_Experiment

para_Executor = {
     'Loader' : {
         'adult'  : {'filepath'  : '../[sunset]/data/[Adt Income] adult.csv'
                    ,'na_values' : {k : '?' for k in ['workclass' ,'occupation' ,'native-country']}
        }
        ,'NHANES' : {'filepath'     : '../[sunset]/data/[NHANES] B.csv'
                    ,'header_exist' : False
                    ,'header_names' : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
        }
     }
    ,'Splitter' : {
         '0.8'  : {'num_samples'       : 2 # 30
                  ,'train_split_ratio' : 0.8
        }
        ,'0.05' : {'num_samples'       : 1 # 30
                  ,'train_split_ratio' : 0.05
        }
    }
    ,'Preprocessor_missing_method'  : 'drop'
    ,'Preprocessor_outlier_method'  : 'IQR'
    # ,'Preprocessor_outlier_columns' : {'ignore': ['hours-per-week']}
    ,'Preprocessor_encoding_method' : 'Label'
    # ,'Preprocessor_scaling'         : True
    # ,'Preprocessor_scaling_method'  : 'Standard'
    # ,'Preprocessor_scaling_columns' : {'focus' : 'fnlwgt'}
    ,'Synthesizer': {
        'GaussianCoupula' : {'synthesizing_method' : 'sdv-singletable-gaussiancoupula'}
       ,'CoupulaGAN'      : {'synthesizing_method' : 'sdv-singletable-coupulagan'}
    #    ,'CTGAN'           : {'synthesizing_method' : 'sdv-singletable-ctgan'}
    #    ,'TVAE'            : {'synthesizing_method' : 'sdv-singletable-tvae'}
    }
}

executor = PETs_Experiment.Executor(**para_Executor)
executor.run()
    

Loading:   0%|          | 0/2 [00:00<?, ?it/s]




Loading:  50%|█████     | 1/2 [00:15<00:15, 15.84s/it]

Loading: 100%|██████████| 2/2 [01:08<00:00, 37.65s/it]

Synthesizer (SDV - SingleTable): Metafile loading time: 0.0236 sec.
Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN.
Synthesizer (SDV - SingleTable): Metafile loading time: 0.108 sec.
Synthesizer (SDV - SingleTable): Fitting  GaussianCoupula.


c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'bmi'. Data will not be rounded.
  warnings.warn(
c:\Program Files\Python310\lib\site-packages\rdt\transformers\numerical.py:112: UserWarning: No rounding scheme detected for column 'bmi'. Data will not be rounded.
  warnings.warn(


...

Synthesizing:   0%|          | 1/240 [04:19<17:13:06, 259.36s/it]

Synthesizer (SDV - SingleTable): Fitting  CoupulaGAN spent 27.087 sec.


Synthesizer (SDV - SingleTable): Sampling CoupulaGAN # 109 rows (same as raw) in 0.2554 sec.
Executor - Synthesizer: CoupulaGAN synthesizing time: 53.7021 sec.


Loading: 100%|██████████| 2/2 [04:44<00:00, 142.40s/it]


====== ====== ====== ====== ====== ======
Executor: Total execution time: 285.8287 sec.
====== ====== ====== ====== ====== ======


In [1]:
executor

NameError: name 'executor' is not defined